# Policy Filter Conversion from C7N YAML to CEL

Wait, we can do that?

Yes. For many (but not all) policies, we can convert the C7N YAML to CEL.

How does it work?

Show me examples.

How do I test it?

## The `xlate.c7n_to_cel` Tool

In [14]:
from xlate.c7n_to_cel import C7N_Rewriter

In [15]:
policy_text = """
name: elb-delete-any-new
resource: elb
comment: 'Any ELB created in any us-west-2 VPC will be immediately deleted, with some
  ASV exceptions.

  '
filters:
- key: CreatedTime
  op: greater-than
  type: value
  value: 0.011
  value_type: age
- key: VPCId
  op: in
  type: value
  value:
  - vpc-eee3698b
  - vpc-66eea803
- key: tag:ASV
  op: not-in
  type: value
  value:
  - ASVZABBIX
  - ASVPUBLICCLOUDBASTIONHOSTS
  - ASVAWSAMI
  - ASVCLOUDOSIMAGES
  - ASVAWSPLATFORM
  - ASVCLOUDCUSTODIAN
  - ASVDATABUNKER
actions:
- key: custodian_decom
  type: tag
  value: Load Balancer created in VPC that is being decommissioned - ELB deleted
- type: delete
- action_descs:
  - The ELB has been deleted.
  - ' '
  - The West VPC of the cof-coaf-dev account is being decommissioned.  Create any
    new resources in the cof-coaf-dev (Dev) East VPC or, for multi-region testing,
    in the West VPC of the cof-consolidatedfs-dev (QA) account, as appropriate.  Also,
    do not use the Default VPC in any environment.
  cc:
  - FSCustodianNotifications@capitalone.com
  from: FSCustodianNotifications@capitalone.com
  policy_url: NA
  subject: '[custodian][{{ account }}] ELB in unallowed account/VPC - {{ region }}'
  template: fs-default.html
  to:
  - resource-owner
  transport:
    topic: arn:aws:sns:{region}:123456789012:c7n-notifications
    type: sns
  type: notify
  violation_desc: 'The following Load Balancer(s) were created in a VPC that is not
    allowed or is being decommissioned:'
"""

In [16]:
print(C7N_Rewriter.c7n_rewrite(policy_text))

Now - duration("15m50s") > timestamp(Resource["CreatedTime"]) && ['vpc-eee3698b', 'vpc-66eea803'].contains(Resource["VPCId"]) && ! ['ASVZABBIX', 'ASVPUBLICCLOUDBASTIONHOSTS', 'ASVAWSAMI', 'ASVCLOUDOSIMAGES', 'ASVAWSPLATFORM', 'ASVCLOUDCUSTODIAN', 'ASVDATABUNKER'].contains(Resource["Tags"].filter(x, x["Key"] == "ASV")[0]["Value"])


That's awkward, lets' reformat it.

We'll be able to replace it in the Policy

    - filters:
      -  type: cel
         expr: |
            Now - duration("15m50s") > timestamp(Resource["CreatedTime"]) 
            && ['vpc-eee3698b', 'vpc-66eea803'].contains(Resource["VPCId"]) 
            && ! ['ASVZABBIX', 'ASVPUBLICCLOUDBASTIONHOSTS', 'ASVAWSAMI', 'ASVCLOUDOSIMAGES', 'ASVAWSPLATFORM', 'ASVCLOUDCUSTODIAN', 'ASVDATABUNKER']
            .contains(Resource["Tags"].filter(x, x["Key"] == "ASV")[0]["Value"])

Which raises an interesting question. 

"15m50s"? Should that be "15m"?  The `0.011` isn't really as accurate as the CEL durations.

The `Resource["Tags"].filter(x, x["Key"] == "ASV")[0]["Value"])` is not "clear" but it's ultra-precise.

Filter all the values in the `Resource["Tags"]` to create a sub-list where the key is "ASV". Ideally, there's exactly one. Item 0 from this list should have a `["Value"]` which is what we want to examine.

This is a common-enough constract, that we will have a extension function for it: `Resource.key("ASV")`. We'll work without the `c7nlib` extensions to start.

In [18]:
cel = """
Now - duration("15m50s") > timestamp(Resource["CreatedTime"]) 
&& ['vpc-eee3698b', 'vpc-66eea803'].contains(Resource["VPCId"]) 
&& ! ['ASVZABBIX', 'ASVPUBLICCLOUDBASTIONHOSTS', 'ASVAWSAMI', 'ASVCLOUDOSIMAGES', 'ASVAWSPLATFORM', 'ASVCLOUDCUSTODIAN', 'ASVDATABUNKER']
    .contains(Resource["Tags"].filter(x, x["Key"] == "ASV")[0]["Value"])
"""

Let's define (or query) some ELB resources and evaluate this CEL expression.

In [19]:
import celpy
decls = {
    "Resource": celpy.celtypes.MapType,
    "Now": celpy.celtypes.TimestampType,
}
env = celpy.Environment(annotations=decls)
ast = env.compile(cel)
prgm = env.program(ast)

In [28]:
description = {
    "ResourceType": "elb",
    "Tags": [
        {"Key": "ASV", "Value": "ASVSOMEAPP"},
    ],
    "CreatedTime": "2020-10-17T18:15:00Z",
    "VPCId": "vpc-66eea803",
}

At 18:19, is this ready for action? (Hint: no, it's not old enough.)

In [31]:
activation_1 = {
    "Resource": celpy.adapter.json_to_cel(description),
    "Now": celpy.celtypes.TimestampType("2020-10-17T18:19:20Z"),
}
result = prgm.evaluate(activation_1)
result

BoolType(False)

How about at 19:20? Now it's been over an hour

In [32]:
activation_2 = {
    "Resource": celpy.adapter.json_to_cel(description),
    "Now": celpy.celtypes.TimestampType("2020-10-17T19:20:21Z"),
}
result = prgm.evaluate(activation_2)
result

BoolType(True)

## More Testing Goodness: the `demo/celdemo.py` tool

How can we more fully automate this testing?

There are two paths.

-  A little shell-level thing to do CEL evaluation in the context of AWS CLI describes.

-  A behave-based framework to do more formal acceptance-type tests.

See the `celdemo.py` script.

(Note. `celpy` is NOT installed, and needs to be visible to Python, using `PYTHONPATH` lets us
use the package without installing it.)

In [12]:
!export PYTHONPATH=src 

In [13]:
!python celdemo.py --cel '355./113.'

3.1415929203539825


In [14]:
!python celdemo.py --cel 'Now+duration("1h")' --now "2020-09-10T11:12:13Z"

2020-09-10T12:12:13Z


This tool creates a CEL activation with two globals.

-  `Now` is set from the command-line `--now` value. Pick a time of day to test against.

-  `Resource` is each resource read from a file or stdin.

You can provide a bunch of resources to apply against a CEL expression and see what the results will be.

You can, for example, use `aws cli` to describe resources and examine them with a `CEL` expression to locate compliant and non-compliant resources.

In [23]:
sample_doc_1 = {
    "Tags": 
    [
        {
            "Key": "ASV",
            "Value": "Forbidden"
        }
    ]
}
from pathlib import Path
import json
with Path("resource1.json").open("w") as rsrc1:
    json.dump(sample_doc_1, rsrc1)

In [33]:
!python celdemo.py --cel 'Resource.Tags.filter(t, t["Key"]=="ASV")[0]["Value"]' --now "2020-09-10T11:12:13Z" resource1.json

Forbidden


In [36]:
!cat resource1.json | python celdemo.py --cel 'Resource.Tags.filter(t, t["Key"]=="ASV")[0]["Value"]' --now "2020-09-10T11:12:13Z" --format json -

Forbidden


In [43]:
!python celdemo.py --cel 'Resource.Tags.filter(t, t["Key"]=="ASV")[0]["Value"]' --now "2020-09-10T11:12:13Z" *.json

TypeError("'CELEvalError' object is not iterable") from Now '2020-09-10T11:12:13Z', Resource {'creationTimestamp': '2018-07-06T05:04:03Z', 'deleteProtection': False, 'name': 'projects/project-123/zones/us-east1-b/instances/dev/ec2', 'instanceSize': 'm1.standard'}
StringType('Forbidden') from Now '2020-09-10T11:12:13Z', Resource {'Tags': [{'Key': 'ASV', 'Value': 'Forbidden'}]}
